# **Instructions**:

### 1 - Open your game that you want to perform detections
### 2 - In the game window, get the name of it's title bar 
### 3 - Update the variable "window_name" with the game title bar name
### 4 - Run all cells to start detecting objects using your trained model

In [1]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import time
import cv2 as cv
import os
import random
import pyautogui
import keyboard

In [2]:
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

 
            
    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        # Bring the window to the front
        self.bring_window_to_front(self.hwnd)

        # Get the window dimensions
        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        # Adjust for borders and title bar
        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

        # Store the window's position on the screen
        self.window_x = window_rect[0] + self.cropped_x
        self.window_y = window_rect[1] + self.cropped_y

    def bring_window_to_front(self, hwnd):
        """Bring the specified window to the front."""
        win32gui.ShowWindow(hwnd, win32con.SW_RESTORE)  # Restore the window if minimized
        time.sleep(0.1)  # Small delay for window to restore
        win32gui.SetWindowPos(hwnd, win32con.HWND_TOPMOST, 0, 0, 0, 0, win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)
        win32gui.SetWindowPos(hwnd, win32con.HWND_NOTOPMOST, 0, 0, 0, 0, win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)
        time.sleep(0.1)  # Small delay for window focus
        
    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img) 
            
        return img
        
    def get_rawshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.frombuffer(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img) 
            
        return img

    def generate_image_dataset(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        while(True):
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.jpeg")
            sleep(1)
    
    def get_window_size(self):
        return (self.w, self.h)


    
    def click_in_window(self,cx, cy, button='left'):
        """
        Perform a click at a relative position within the window.
        :param rel_x: X coordinate relative to the window.
        :param rel_y: Y coordinate relative to the window.
        :param button: Mouse button to click ('left', 'right', 'middle').
        """
        
        # Calculate the absolute screen coordinates
        abs_x = self.window_x + cx
        abs_y = self.window_y + cy
    
        # # Move the mouse and click
        pyautogui.moveTo(abs_x, abs_y, duration=0.05)
        pyautogui.mouseDown()
        pyautogui.mouseUp()

In [3]:
class ImageProcessor:
    W = 0
    H = 0
    net = None
    ln = None
    classes = {}
    colors = []

    def __init__(self, img_size, cfg_file, weights_file):
        np.random.seed(42)
        self.net = cv.dnn.readNetFromDarknet(cfg_file, weights_file)
        self.net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.ln = self.net.getLayerNames()
        self.ln = [self.ln[i-1] for i in self.net.getUnconnectedOutLayers()]
        self.W = img_size[0]
        self.H = img_size[1]
        
        with open('yolov4-tiny/obj.names', 'r') as file:
            lines = file.readlines()
        for i, line in enumerate(lines):
            self.classes[i] = line.strip()
        
        # If you plan to utilize more than six classes, please include additional colors in this list.
        self.colors = [
            (0, 0, 255), 
            (0, 255, 0), 
            (255, 0, 0), 
            (255, 255, 0), 
            (255, 0, 255), 
            (0, 255, 255),
            (128, 0, 128),
            (128, 0, 128)
        ]
        

    def proccess_image(self, img):

        blob = cv.dnn.blobFromImage(img, 1/255.0, (320, 320), swapRB=True, crop=False)
        self.net.setInput(blob)
        outputs = self.net.forward(self.ln)
        outputs = np.vstack(outputs)
        
        coordinates = self.get_coordinates(outputs, 0.5)

        self.draw_identified_objects(img, coordinates)

        return coordinates

    def get_coordinates(self, outputs, conf):

        boxes = []
        confidences = []
        classIDs = []

        for output in outputs:
            scores = output[5:]
            
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > conf:
                x, y, w, h = output[:4] * np.array([self.W, self.H, self.W, self.H])
                p0 = int(x - w//2), int(y - h//2)
                boxes.append([*p0, int(w), int(h)])
                confidences.append(float(confidence))
                classIDs.append(classID)

        indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)

        if len(indices) == 0:
            return []

        coordinates = []
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            coordinates.append({'x': x, 'y': y, 'w': w, 'h': h, 'class': classIDs[i], 'class_name': self.classes[classIDs[i]]})
        return coordinates

    def draw_identified_objects(self, img, coordinates):
        for coordinate in coordinates:
            x = coordinate['x']
            y = coordinate['y']
            w = coordinate['w']
            h = coordinate['h']
            classID = coordinate['class']
            
            color = self.colors[classID]
            
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, self.classes[classID], (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv.imshow('window',  img)


In [11]:
# # Run this cell to initiate detections using the trained model.

# window_name = "MixMaster Online"
# cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
# weights_file_name = "yolov4-tiny-custom_last.weights"

# wincap = WindowCapture(window_name)
# improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)
# def find_core():
#     wincap = WindowCapture(window_name)
#     # while(True):
#         # wincap = WindowCapture(window_name)
#     ss = wincap.get_screenshot()
    
#     # if cv.waitKey(1) == ord('q'):
#     #     cv.destroyAllWindows()
#     #     break

#     coordinates = improc.proccess_image(ss)
    
#     if coordinates:
#         for coordinate in coordinates:
#             print(coordinate)
#         print()
#         w_num = coordinates[0]['w']
#         h_num = coordinates[0]['h']
#         x_num = coordinates[0]['x']
#         y_num = coordinates[0]['y']
#         cx = x_num + (w_num / 2)
#         cy = y_num + (h_num / 2)
        
#         wincap.click_in_window(cx, cy)
#         # print(f"The x:{coordinates[0]['w']}, The y:{coordinates[0]['h']}")
#         # print(f"The type of x is:{type(w_num)} and y is :{type(h_num)}")
#         # If you have limited computer resources, consider adding a sleep delay between detections.
#         # sleep(5)
#     else:
#         print('No coordinates found. The list is empty.')

# print('Finished.')



# Run this cell to initiate detections using the trained model.

# window_name = "MixMaster Online"
# cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
# weights_file_name = "yolov4-tiny-custom_last.weights"

# wincap = WindowCapture(window_name)
# improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)
# def find_core():
#     wincap = WindowCapture(window_name)
#     # while(True):
#         # wincap = WindowCapture(window_name)
#     ss = wincap.get_screenshot()
#     previous_coordinates = improc.proccess_image(ss)
#     # if cv.waitKey(1) == ord('q'):
#     #     cv.destroyAllWindows()
#     #     break
#     ss = wincap.get_screenshot()
#     newest_coordinates = improc.proccess_image(ss)

#     if previous_coordinates and newest_coordinates:
#          if previous_coordinates[0] != newest_coordinates[0]:
#             w_num = newest_coordinates[0]['w']
#             h_num = newest_coordinates[0]['h']
#             x_num = newest_coordinates[0]['x']
#             y_num = newest_coordinates[0]['y']

#             predicted_value_x = x_num + x_num * 0.1
#             predicted_value_y = y_num + y_num * 0.1

#             cx = predicted_value_x + (w_num / 2)
#             cy = predicted_value_y + (h_num / 2)
            
#             wincap.click_in_window(cx, cy)

#          else:
#             w_num = newest_coordinates[0]['w']
#             h_num = newest_coordinates[0]['h']
#             x_num = newest_coordinates[0]['x']
#             y_num = newest_coordinates[0]['y']
#             cx = x_num + (w_num / 2)
#             cy = y_num + (h_num / 2)
            
#             wincap.click_in_window(cx, cy)
#             # print(f"The x:{coordinates[0]['w']}, The y:{coordinates[0]['h']}")
#             # print(f"The type of x is:{type(w_num)} and y is :{type(h_num)}")
#             # If you have limited computer resources, consider adding a sleep delay between detections.
#             # sleep(5)
#     else:
#         print('No coordinates found. The list is empty.')

# print('Finished.')


window_name = "MixMaster Online"
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
weights_file_name = "yolov4-tiny-custom_last.weights"

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)

previous_coordinates = []  # Store only living objects
velocity_multiplier = 0.1  # Adjust prediction speed

def find_core():
    global previous_coordinates  
    wincap = WindowCapture(window_name)
    ss = wincap.get_screenshot()
    newest_coordinates = improc.proccess_image(ss)

    if newest_coordinates:
        # Find the closest living object (not previously clicked)
        best_match = None
        min_distance = float("inf")

        for new in newest_coordinates:
            for prev in previous_coordinates:
                # Compute distance between objects
                dist = ((new['x'] - prev['x']) ** 2 + (new['y'] - prev['y']) ** 2) ** 0.5
                if dist < min_distance:
                    min_distance = dist
                    best_match = (prev, new)

        if best_match:
            prev_obj, new_obj = best_match
            # Calculate velocity
            velocity_x = (new_obj['x'] - prev_obj['x'])
            velocity_y = (new_obj['y'] - prev_obj['y'])

            # Predict next position
            predicted_x = new_obj['x'] + velocity_x * velocity_multiplier
            predicted_y = new_obj['y'] + velocity_y * velocity_multiplier
        else:
            # No previous match, just use the first detected object
            predicted_x = newest_coordinates[0]['x']
            predicted_y = newest_coordinates[0]['y']

        # Get object size
        w_num = newest_coordinates[0]['w']
        h_num = newest_coordinates[0]['h']

        # Click in the predicted center
        cx = predicted_x + (w_num / 2)
        cy = predicted_y + (h_num / 2)

        wincap.click_in_window(cx, cy)

        # Store only objects that were NOT clicked (alive objects)
        previous_coordinates = [obj for obj in newest_coordinates if obj['x'] != predicted_x and obj['y'] != predicted_y]
    else:
        print("No coordinates found. The list is empty.")

print("Finished.")


Finished.


In [5]:
def capture_window(window_name):
    hwnd = win32gui.FindWindow(None, window_name)
    if hwnd:
        rect = win32gui.GetWindowRect(hwnd)
        w = rect[2] - rect[0]
        h = rect[3] - rect[1]

        hdc = win32gui.GetWindowDC(hwnd)
        hdc_mem = win32ui.CreateDCFromHandle(hdc)
        hdc_compatible = hdc_mem.CreateCompatibleDC()

        bitmap = win32ui.CreateBitmap()
        bitmap.CreateCompatibleBitmap(hdc_mem, w, h)
        hdc_compatible.SelectObject(bitmap)

        hdc_compatible.BitBlt((0, 0), (w, h), hdc_mem, (0, 0), win32con.SRCCOPY)

        signed_ints_array = bitmap.GetBitmapBits(True)
        img = np.frombuffer(signed_ints_array, dtype='uint8')
        img.shape = (h, w, 4)

        win32gui.DeleteObject(bitmap.GetHandle())
        hdc_mem.DeleteDC()
        hdc_compatible.DeleteDC()
        win32gui.ReleaseDC(hwnd, hdc)

        return img[..., :3]
    else:
        print("Window not found!")
        return None

In [6]:
# Capture the "Mixmaster Online" window
window_name = "Mixmaster Online"

template = cv.imread('img/healthbar2.png')  # Load the template image
# Check if the template is loaded properly
def timer():
    if template is None:
        print("Failed to load template image!")
    else:
        while True:
            # Display the template image for reference
            # cv.imshow('Template', template)
    
            # Capture the window image
            raw_img = capture_window(window_name)
    
            if raw_img is not None:
                # Define the specific region (location and size) based on the provided information
                x, y = 485, 86  # Coordinates for the top-left corner of the region
                w, h = 62, 11  # Width and height of the template (as per the provided size)
    
                # Crop the captured image to the region of interest (ROI)
                roi = raw_img[y:y+h, x:x+w]
    
                # Make a writable copy of the cropped region (ROI)
                raw_img_copy = np.copy(roi)
    
                # Display the captured raw image (ROI)
                # cv.imshow('Captured ROI Image', raw_img_copy)
    
                # Perform template matching on the cropped region (ROI)
                result = cv.matchTemplate(raw_img_copy, template, cv.TM_CCOEFF_NORMED)
    
                # Get min, max values and their locations
                min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
    
                # Get the height and width of the template
                # print(f"Template height: {h}, Template width: {w}")
                # print(f"Match location: {max_loc}")
                # print(f"Max value: {max_val}")
    
                # Draw a rectangle around the matched region on the cropped region
                top_left = max_loc
                bottom_right = (top_left[0] + w, top_left[1] + h)
                cv.rectangle(raw_img_copy, top_left, bottom_right, (0, 255, 0), 2)
                return max_val
                # Show the matched image
                # cv.imshow('Matched ROI Image', raw_img_copy)
    
                # Wait for key press and close the windowsq
                if cv.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                print("Failed to capture the window!")
            
            # Add a small delay to prevent high CPU usage
            time.sleep(2)
        cv.destroyAllWindows()  # Close all windows after exiting the loop

In [10]:
template = cv.imread('img/healthbar2.png')  # Load the template image
cv.imshow('healthbar', template)
cv.waitKey(0)
cv.destroyAllWindows()

In [12]:
clicked = False
while True:
    find_core()
    value = timer()
    if value > 0.5:
        start_time = time.time()
        previous_value = value 
        clicked = True
        while clicked:
            value = timer() 
            print(f"Timer value: {value}")
            if previous_value != value:  
                start_time = time.time()
                previous_value = value
            else:
                if time.time() - start_time >= 1:
                    break
            new_value = timer()
            if new_value < 0.5:
                clicked = False
            # Check if the 'Esc' key is pressed
            if keyboard.is_pressed('esc'):
                print("Esc key pressed. Exiting loop...")
                break  # Exit the inner loop
    else:
        clicked = False
        time.sleep(1)

    # Check if the 'Esc' key is pressed (to exit the outer loop)
    if keyboard.is_pressed('esc'):
        print("Esc key pressed. Exiting program...")
        break  # Exit the outer loop
# clicked = False
# while True:
#     find_core()
#     value = timer()
    
#     if value > 0.5:
#         start_time = time.time()
#         previous_value = value 
#         clicked = True
        
#         while clicked:
#             value = timer() 
#             print(f"Timer value: {value}")
            
#             if previous_value != value:  
#                 start_time = time.time()
#                 previous_value = value
#             else:
#                 if time.time() - start_time >= 1:
#                     break

#             # Track the object if detected
#             ss = wincap.get_screenshot()
#             success, box = tracker.update(ss)  # Update the position of the tracked object
            
#             if success:
#                 # Draw a bounding box around the tracked object
#                 x, y, w, h = [int(v) for v in box]
#                 cv.rectangle(ss, (x, y), (x + w, y + h), (255, 0, 0), 2)
#                 # Optionally, click again based on new position
#                 wincap.click_in_window(x + w / 2, y + h / 2)
            
#             # Exit conditions
#             if keyboard.is_pressed('esc'):
#                 print("Esc key pressed. Exiting loop...")
#                 break
#     else:
#         clicked = False
#         time.sleep(1)

#     if keyboard.is_pressed('esc'):
#         print("Esc key pressed. Exiting program...")
#         break

C:\Users\undang\AppData\Local\Temp\ipykernel_15020\783640516.py:51: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


Timer value: 0.6856617331504822
Timer value: 0.6856617331504822
Timer value: 0.6856617331504822
Timer value: 0.7130717635154724
Timer value: 0.7130717635154724
Timer value: 0.7630969285964966
Timer value: 0.752769947052002
Timer value: 0.752769947052002
Timer value: 0.752769947052002
Timer value: 0.7723708152770996
Timer value: 0.7723708152770996
Timer value: 0.7723708152770996
Timer value: 0.7601993083953857
Timer value: 0.7601993083953857
Timer value: 0.7509497404098511
Timer value: 0.7509497404098511
Timer value: 0.7486522197723389
Timer value: 0.7486522197723389
Timer value: 0.7486522197723389
Timer value: 0.7462564706802368
Timer value: 0.7258868217468262
Timer value: 0.7258868217468262
Timer value: 0.7258868217468262
Timer value: 0.7076561450958252
Timer value: 0.7076561450958252
Timer value: 0.7076561450958252
Timer value: 0.7879512310028076
Timer value: 0.7879512310028076
Timer value: 0.8003291487693787
Timer value: 0.8003291487693787
Timer value: 0.8003291487693787
Timer value